This project is an attempt to create a model that can predict the outcome of a tic-tac-toe game, given the state of the board.

In [1]:
# Getting the dataset
import numpy as np
from sklearn.datasets import fetch_openml
# downloaded_dataset = fetch_openml("tic-tac-toe", version=1)
# np.save("dataset.npy", downloaded_dataset)

In [2]:
dataset_original = np.load("dataset.npy", allow_pickle=True)
print(dataset_original)

{'data':     top-left-square top-middle-square top-right-square middle-left-square  \
0                 x                 x                x                  x   
1                 x                 x                x                  x   
2                 x                 x                x                  x   
3                 x                 x                x                  x   
4                 x                 x                x                  x   
..              ...               ...              ...                ...   
953               o                 x                x                  x   
954               o                 x                o                  x   
955               o                 x                o                  x   
956               o                 x                o                  o   
957               o                 o                x                  x   

    middle-middle-square middle-right-square bottom-left-square  \

This dataset contains all possible board states, and the status of each square. There are no features to remove here.
The aim is to train a model that can predict wether the player "x" has lost or won the game.
However, there is work to be done on the values. Characters like "x", "o" or "b" are not useful here, so we will convert each of them to 1,-1 and 0, respectively.

Next, we will separate the values from the labels.

In [3]:
import pandas as pd

dataset = dataset_original.flat[0]['frame'].copy()
values = pd.DataFrame(dataset.loc[:, "top-left-square":"bottom-right-square"])
labels = pd.DataFrame(dataset.loc[:, "Class"])

values, labels

(    top-left-square top-middle-square top-right-square middle-left-square  \
 0                 x                 x                x                  x   
 1                 x                 x                x                  x   
 2                 x                 x                x                  x   
 3                 x                 x                x                  x   
 4                 x                 x                x                  x   
 ..              ...               ...              ...                ...   
 953               o                 x                x                  x   
 954               o                 x                o                  x   
 955               o                 x                o                  x   
 956               o                 x                o                  o   
 957               o                 o                x                  x   
 
     middle-middle-square middle-right-square bottom-left-squa

Now, we will transform the values into ones that are useful to us.

In [5]:
X = values.to_numpy()
function = lambda v: 1 if v == 'x' else(-1 if v == 'o' else 0)
function = np.vectorize(function) # We need to vectorize the function to apply it to the array

X = function(X) # applying the function to the np array
X

array([[ 1,  1,  1, ...,  1, -1, -1],
       [ 1,  1,  1, ..., -1,  1, -1],
       [ 1,  1,  1, ..., -1, -1,  1],
       ...,
       [-1,  1, -1, ...,  1, -1,  1],
       [-1,  1, -1, ...,  1, -1,  1],
       [-1, -1,  1, ..., -1,  1,  1]])

Now we will do a similar operation on the labels.

In [11]:
y = labels.to_numpy()

function = lambda v: 1 if v == 'positive' else 0
function = np.vectorize(function)

y = function(y)

Now, we will divide the train and test set, and proceed with the training.

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  X, y , random_state=42,test_size=0.25, shuffle=True)

y_train = y_train.ravel()
y_test = y_test.ravel()

In [13]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(n_jobs = -1, random_state=42)
sgd_clf.fit(X_train, y_train)

SGDClassifier(n_jobs=-1, random_state=42)

In [14]:
from sklearn.model_selection import cross_val_score


cvs = cross_val_score(
                sgd_clf, 
                X_test, 
                y = y_test,
                cv = 3,
                n_jobs = -1)

print(cvs)

[0.9375 0.9625 0.95  ]


So, the accuracy is between 93.75% and 96.25% on 3 folds.
Below, I calculated the accuracy of a model which would only return True or False.

In [15]:
values, counts = np.unique(y_test, return_counts=True)
total = len(y_test)
percentages = [counts[i]/total * 100 for i in range(len(values))]
print([(values[i], percentages[i]) for i in range(len(values))])

[(np.int64(0), np.float64(31.25)), (np.int64(1), np.float64(68.75))]


So, a "dumb" model would only get a maximum accuracy of 68.75% if it predicted True for every input.

This code dumps the model into a file.

In [17]:
import joblib
joblib.dump(sgd_clf, "model.pkl")

['model.pkl']